# Практическое задание №9 по теме "Языковое моделирование".

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = 'C:/Study/GO.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 394567 characters


In [4]:
print(text[5405:5660])


Автор: Егор Летов
Альбом: Жизнь что сказка


Заброшенное ж-д, заржавелый прогнивший завод,
карликовый городок на горизонте,
бродячие кучи деревьев, живых по привычке,
разумеется, свалка, серые сумерки.
Вот здесь-то меня и убили, а потом не нашли


In [5]:
vocab = sorted(set(text))

In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [10]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [11]:
vocab_size = len(vocab)
embedding_dim = 128
rnn_units = 1024

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [13]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [14]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 146) # (batch_size, sequence_length, vocab_size)


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         18688     
                                                                 
 gru (GRU)                   (None, None, 1024)        3545088   
                                                                 
 gru_1 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_2 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense (Dense)               (None, None, 146)         149650    
                                                                 
Total params: 16,308,626
Trainable params: 16,308,626
Non-trainable params: 0
_________________________________________________________________


In [16]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [17]:
model.compile(optimizer='adam', loss=loss)

In [18]:
checkpoint_dir = 'C:/Study/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

In [19]:
history = model.fit(dataset, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
61/61 [==============================] - 383s 6s/step - loss: 3.2409
Epoch 2/50
61/61 [==============================] - 396s 6s/step - loss: 2.3633
Epoch 3/50
61/61 [==============================] - 394s 6s/step - loss: 2.0985
Epoch 4/50
61/61 [==============================] - 397s 7s/step - loss: 1.9391
Epoch 5/50
61/61 [==============================] - 399s 7s/step - loss: 1.7928
Epoch 6/50
61/61 [==============================] - 398s 7s/step - loss: 1.6484
Epoch 7/50
61/61 [==============================] - 401s 7s/step - loss: 1.5016
Epoch 8/50
61/61 [==============================] - 402s 7s/step - loss: 1.3635
Epoch 9/50
61/61 [==============================] - 405s 7s/step - loss: 1.2326
Epoch 10/50
61/61 [==============================] - 399s 7s/step - loss: 1.1084
Epoch 11/50
61/61 [==============================] - 398s 7s/step - loss: 0.9875
Epoch 12/50
61/61 [==============================] - 399s 7s/step - loss: 0.8676
Epoch 13/50
61/61 [==================

In [20]:
tf.train.latest_checkpoint('./')

In [21]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         18688     
                                                                 
 gru_3 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 gru_4 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_5 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_1 (Dense)             (None, None, 146)         149650    
                                                                 
Total params: 16,308,626
Trainable params: 16,308,626
Non-trainable params: 0
_________________________________________________________________


In [23]:
def generate_text(model, start_string):
  
    num_generate = 500

  
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 1

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [24]:
text_ = generate_text(model, start_string=u"Пелена невозможной ясности бессмертия ")
print(text_)

Пелена невозможной ясности бессмертия —      ки Ле,    намобои-тыгамарнт A Вомежня     иры бисра)   выгаз.
F#,  
Я за ной епупрамы  C#,вся   уся,  кирушазазегамазама  ит  мия,D прте
Am  A,ужамай  м ист
Ле
  Гам     ми
Алиточи
Онечля        стод  кулемо Лерий    нЕва Ленот
Я   в, пои      ре.
Ончемет коли кавортоз  ри      ой пу
ЗИ виснь
   сяс     мато

В з  ироеборежамизо   G  A :   ра       —   азваразато ол
И пи, мартах F#m   
Летвобомый  ко   о
Прасазуказм тыми
Бо А   
3 Димеса F            Em    раздетои, лто


---